In [1]:
!pip install pandas
!pip install pypdf
!pip install pypdf2
!pip install cryptography
!pip install pikepdf

  Using cached pypdf-5.4.0-py3-none-any.whl.metadata (7.3 kB)
Using cached pypdf-5.4.0-py3-none-any.whl (302 kB)


In [ ]:
import os
import shutil
import pikepdf
import re
import pandas as pd
from PyPDF2 import PdfReader

# Caminhos das pastas
source_folder = "./maps/encrypted/"
processed_folder = os.path.join(source_folder, "processed")
decrypted_folder = "./maps/decrypted/"
decrypted_processed_folder = os.path.join(decrypted_folder, "processed")

def ensure_folders_exist():
    folders = [source_folder, processed_folder, decrypted_folder, decrypted_processed_folder]
    for folder in folders:
        os.makedirs(folder, exist_ok=True)

ensure_folders_exist()

# Descriptografar PDF
pdf_files = [f for f in os.listdir(source_folder) if f.endswith('.pdf')]
if not pdf_files:
    print("Nenhum PDF encontrado.")
else:
    encrypted_pdf_path = os.path.join(source_folder, pdf_files[0])
    decrypted_pdf_path = os.path.join(decrypted_folder, f"decrypted_{pdf_files[0]}")
    try:
        with pikepdf.open(encrypted_pdf_path) as pdf:
            pdf.save(decrypted_pdf_path)
        print(f"PDF desbloqueado com sucesso: {decrypted_pdf_path}")
        shutil.move(encrypted_pdf_path, os.path.join(processed_folder, pdf_files[0]))
    except Exception as e:
        print(f"Erro ao desbloquear: {e}")

# Processamento do PDF
input_folder = "./maps/decrypted"
processed_folder = "./maps/decrypted/processed"
os.makedirs(processed_folder, exist_ok=True)
pdf_files = [file for file in os.listdir(input_folder) if file.lower().endswith(".pdf")]

texto_blocos = {}
if pdf_files:
    for file_name in pdf_files:
        input_pdf_path = os.path.join(input_folder, file_name)
        processed_pdf_path = os.path.join(processed_folder, file_name)
        try:
            full_text = ""
            reader = PdfReader(input_pdf_path)
            for idx, page in enumerate(reader.pages):
                full_text += page.extract_text() + "\n"

            print(f"\nPDF: {file_name} | Total de páginas: {len(reader.pages)}")

            # Separar por instituições
            instituicoes = re.split(
                r"(Informação comunicada pela instituição:.+?(?=Informação comunicada pela instituição:|$))",
                full_text,
                flags=re.DOTALL
            )

            idx_geral = 1
            for inst_text in instituicoes:
                inst_match = re.search(r"Informação comunicada pela instituição:\s+(.+)", inst_text)
                nome_inst = inst_match.group(1).strip() if inst_match else "NÃO IDENTIFICADA"

                blocos = re.findall(
                    r"(Montantes.*?Produto financeiro.+?)(?=Montantes|Informação comunicada pela instituição:|$)",
                    inst_text,
                    flags=re.DOTALL
                )

                for bloco in blocos:
                    bloco_completo = f"Informação comunicada pela instituição: {nome_inst}\n{bloco.strip()}"
                    texto_blocos[f"produto_{idx_geral}"] = bloco_completo
                    idx_geral += 1

            print(f"Total de produtos detectados em '{file_name}': {len(texto_blocos)}")

        except Exception as e:
            print(f"Erro ao processar '{file_name}': {e}")
        finally:
            shutil.move(input_pdf_path, processed_pdf_path)
            print(f"PDF '{file_name}' movido para: {processed_pdf_path}")
else:
    print("Nenhum PDF encontrado na pasta de entrada.")

# Regexes
regexes = {
    'nome': re.compile(r'Nome:\s+(.+)', re.MULTILINE),
    'nif': re.compile(r'Nº de Identificação:\s+(\d+)'),
    'mes_mapa': re.compile(r'Responsabilidades de crédito referentes a\s+(.+)'),
    'instituicao': re.compile(r'Informação comunicada pela instituição:\s+(.+)'),
    'total_em_divida': re.compile(r"Total em dívida\s+do qual, em incumprimento\s+([\d\s,]+) €"),
    'litigio': re.compile(r'Em litígio judicial\s+(Sim|Não)'),
    'abatido_ativo': re.compile(r'Abatido ao ativo\s+([\d\s,.]+) €'),
    'garantias': re.compile(r"Tipo\s+Valor\s+Número[\s\S]+?([\d\s,.]+) €"),
    'num_devedores': re.compile(r"Nº devedores no contrato\s+(\d+)"),
    'prod_financeiro': re.compile(r"Produto financeiro\s+(.+?)\s+Tipo de responsabilidade"),
    'dat_inicio': re.compile(r"Início\s+(\d{4}-\d{2}-\d{2})"),
    'dat_fim': re.compile(r"Fim\s+(\d{4}-\d{2}-\d{2})"),
    'entrada_incumpr': re.compile(r"Entrada incumpr\.\s+(\d{4}-\d{2}-\d{2})")
}

def get_feature(text, regex_string):
    match = regexes[regex_string].search(text)
    return match.group(1).strip() if match else None

# Construção do DataFrame
data = []
for bloco_id, bloco_text in texto_blocos.items():
    row = {
        'bloco_id': bloco_id,
        'nome': get_feature(bloco_text, 'nome'),
        'nif': get_feature(bloco_text, 'nif'),
        'mes_mapa': get_feature(bloco_text, 'mes_mapa'),
        'instituicao': get_feature(bloco_text, 'instituicao'),
        'divida': get_feature(bloco_text, 'total_em_divida'),
        'litigio': get_feature(bloco_text, 'litigio'),
        'parcela': get_feature(bloco_text, 'abatido_ativo'),
        'garantias': get_feature(bloco_text, 'garantias'),
        'num_devedores': get_feature(bloco_text, 'num_devedores'),
        'prod_financeiro': get_feature(bloco_text, 'prod_financeiro'),
        'entrada_incumpr': get_feature(bloco_text, 'entrada_incumpr'),
        'dat_inicio': get_feature(bloco_text, 'dat_inicio'),
        'dat_fim': get_feature(bloco_text, 'dat_fim')
    }
    data.append(row)

df = pd.DataFrame(data)
df

PDF desbloqueado com sucesso: ./maps/decrypted/decrypted_ea372f68-db7d-4148-ae6b-598ac781f106.pdf

PDF: decrypted_ea372f68-db7d-4148-ae6b-598ac781f106.pdf | Total de páginas: 5
Total de produtos detectados em 'decrypted_ea372f68-db7d-4148-ae6b-598ac781f106.pdf': 5
PDF 'decrypted_ea372f68-db7d-4148-ae6b-598ac781f106.pdf' movido para: ./maps/decrypted/processed/decrypted_ea372f68-db7d-4148-ae6b-598ac781f106.pdf


,bloco_id,nome,nif,mes_mapa,instituicao,divida,litigio,parcela,garantias,num_devedores,prod_financeiro,entrada_incumpr,dat_inicio,dat_fim
0,produto_1,ALDINA MARIA PARDAL BALTAZAR MARTINS,195129180,agosto de 2023,"CAIXA GERAL DE DEPÓSITOS, S.A. (0035)","161,78",Não,"0,00","0,00",2,None,None,2009-05-08,9999-12-31
1,produto_2,None,None,None,"BANCO COMERCIAL PORTUGUÊS, SA (0033)","0,00",Não,"0,00","0,00",1,Cartão de crédito - com período de free-float,None,2016-08-10,9999-12-31
2,produto_3,ALDINA MARIA PARDAL BALTAZAR MARTINS,195129180,agosto de 2023,"BANCO COMERCIAL PORTUGUÊS, SA (0033)","476,43",Não,"0,00","0,00",2,None,None,2013-03-01,9999-12-31
3,produto_4,ALDINA MARIA PARDAL BALTAZAR MARTINS,195129180,agosto de 2023,COFIDIS (0921),"0,00",Não,"0,00","0,00",2,Crédito renovável - Linha de crédito,None,2006-07-19,9999-12-31
4,produto_5,ALDINA MARIA PARDAL BALTAZAR MARTINS,195129180,agosto de 2023,"BNP PARIBAS PERSONAL FINANCE, S.A. - SUCURSAL EM PORTUGAL (0848)","27 536,59",Não,"562,71","562,71",2,Crédito pessoal,None,2022-09-22,2029-10-01


In [152]:
import os
import shutil
import pikepdf
import re
import pandas as pd
from PyPDF2 import PdfReader
import unicodedata
from IPython.display import display

# Caminhos das pastas
source_folder = "./maps/encrypted/"
processed_folder = os.path.join(source_folder, "processed")
decrypted_folder = "./maps/decrypted/"
decrypted_processed_folder = os.path.join(decrypted_folder, "processed")

def ensure_folders_exist():
    folders = [source_folder, processed_folder, decrypted_folder, decrypted_processed_folder]
    for folder in folders:
        os.makedirs(folder, exist_ok=True)

ensure_folders_exist()

# Descriptografar PDF
pdf_files = [f for f in os.listdir(source_folder) if f.endswith('.pdf')]
if not pdf_files:
    print("Nenhum PDF encontrado.")
else:
    encrypted_pdf_path = os.path.join(source_folder, pdf_files[0])
    decrypted_pdf_path = os.path.join(decrypted_folder, f"decrypted_{pdf_files[0]}")
    try:
        with pikepdf.open(encrypted_pdf_path) as pdf:
            pdf.save(decrypted_pdf_path)
        print(f"PDF desbloqueado com sucesso: {decrypted_pdf_path}")
        shutil.move(encrypted_pdf_path, os.path.join(processed_folder, pdf_files[0]))
    except Exception as e:
        print(f"Erro ao desbloquear: {e}")

# Processamento do PDF
input_folder = "./maps/decrypted"
processed_folder = "./maps/decrypted/processed"
os.makedirs(processed_folder, exist_ok=True)
pdf_files = [file for file in os.listdir(input_folder) if file.lower().endswith(".pdf")]

texto_blocos = {}
if pdf_files:
    for file_name in pdf_files:
        input_pdf_path = os.path.join(input_folder, file_name)
        processed_pdf_path = os.path.join(processed_folder, file_name)
        try:
            full_text = ""
            reader = PdfReader(input_pdf_path)
            for idx, page in enumerate(reader.pages):
                full_text += page.extract_text() + "\n"

            print(f"\nPDF: {file_name} | Total de páginas: {len(reader.pages)}")

            instituicoes = re.split(
                r"(Informação comunicada pela instituição:.+?(?=Informação comunicada pela instituição:|$))",
                full_text,
                flags=re.DOTALL
            )

            idx_geral = 1
            for inst_text in instituicoes:
                inst_match = re.search(r"Informação comunicada pela instituição:\s+(.+)", inst_text)
                nome_inst = inst_match.group(1).strip() if inst_match else "NÃO IDENTIFICADA"

                blocos = re.findall(
                    r"(Montantes.*?Produto financeiro.+?)(?=Montantes|Informação comunicada pela instituição:|$)",
                    inst_text,
                    flags=re.DOTALL
                )

                for bloco in blocos:
                    bloco_completo = f"Informação comunicada pela instituição: {nome_inst}\n{bloco.strip()}"
                    texto_blocos[f"produto_{idx_geral}"] = bloco_completo
                    idx_geral += 1

            print(f"Total de produtos detectados em '{file_name}': {len(texto_blocos)}")

        except Exception as e:
            print(f"Erro ao processar '{file_name}': {e}")
        finally:
            shutil.move(input_pdf_path, processed_pdf_path)
            print(f"PDF '{file_name}' movido para: {processed_pdf_path}")
else:
    print("Nenhum PDF encontrado na pasta de entrada.")

# Regexes
regexes = {
    'nome': re.compile(r'Nome:\s+(.+)', re.MULTILINE),
    'nif': re.compile(r'Nº de Identificação:\s+(\d+)'),
    'mes_mapa': re.compile(r'Responsabilidades de crédito referentes a\s+(.+)'),
    'instituicao': re.compile(r'Informação comunicada pela instituição:\s+(.+)'),
    'total_em_divida': re.compile(r"Total em dívida\s+do qual, em incumprimento\s+([\d\s\u00A0,.]+) €"),
    'litigio': re.compile(r'Em litígio judicial\s+(Sim|Não)'),
    'abatido_ativo': re.compile(r'Abatido ao ativo\s+([\d\s\u00A0,.]+) €'),
    'garantias': re.compile(r"Tipo\s+Valor\s+Número\s*\n(?:.*?\n)?\d+\s+([\d\s\u00A0,.]+) €"),
    'num_devedores': re.compile(r"Nº devedores no contrato\s+(\d+)"),
    'prod_financeiro': re.compile(r"Produto financeiro\s+(.+?)\s+Tipo de responsabilidade"),
    'dat_inicio': re.compile(r"Início\s+(\d{4}-\d{2}-\d{2})"),
    'dat_fim': re.compile(r"Fim\s+(\d{4}-\d{2}-\d{2})"),
    'entrada_incumpr': re.compile(r"Entrada incumpr\.\s+(\d{4}-\d{2}-\d{2})")
}

def get_feature(text, regex_string):
    match = regexes[regex_string].search(text)
    return match.group(1).strip() if match else None

# Construção do DataFrame
data = []
for bloco_id, bloco_text in texto_blocos.items():
    row = {
        'bloco_id': bloco_id,
        'nome': get_feature(bloco_text, 'nome'),
        'nif': get_feature(bloco_text, 'nif'),
        'mes_mapa': get_feature(bloco_text, 'mes_mapa'),
        'instituicao': get_feature(bloco_text, 'instituicao'),
        'divida': get_feature(bloco_text, 'total_em_divida'),
        'litigio': get_feature(bloco_text, 'litigio'),
        'parcela': get_feature(bloco_text, 'abatido_ativo'),
        'garantias': get_feature(bloco_text, 'garantias'),
        'num_devedores': get_feature(bloco_text, 'num_devedores'),
        'prod_financeiro': get_feature(bloco_text, 'prod_financeiro'),
        'entrada_incumpr': get_feature(bloco_text, 'entrada_incumpr'),
        'dat_inicio': get_feature(bloco_text, 'dat_inicio'),
        'dat_fim': get_feature(bloco_text, 'dat_fim')
    }
    data.append(row)

df = pd.DataFrame(data)

# Conversão de valores monetários
def parse_float(valor_str):
    if isinstance(valor_str, str):
        valor_str = ''.join(c for c in valor_str if not unicodedata.category(c).startswith('Z'))
        valor_str = valor_str.replace('€', '').replace(',', '.')
        try:
            return float(valor_str)
        except ValueError:
            return None
    return None

for col in ['divida', 'parcela', 'garantias']:
    df[col] = df[col].apply(parse_float)

# ... (todo o código anterior até a criação do DataFrame e parse_float permanece igual)

# Regras de perfilamento individual
def regra_sem_garantia(row):
    return pd.isna(row['garantias']) or row['garantias'] == 0.0

def regra_sem_litigio(row):
    return isinstance(row['litigio'], str) and row['litigio'].strip().lower() == 'não'

instituicoes_com_garantia = df[df['garantias'] > 0]['instituicao'].unique().tolist()

def instituicao_sem_qualquer_garantia(row):
    return row['instituicao'] not in instituicoes_com_garantia

# 1️⃣ Define perfil_individual por linha
df['perfil_individual'] = df.apply(
    lambda r: regra_sem_garantia(r) and regra_sem_litigio(r) and instituicao_sem_qualquer_garantia(r),
    axis=1
)

# 2️⃣ Soma global de dívidas válidas
soma_total_elegivel = df[df['perfil_individual']]['divida'].sum()

# 3️⃣ Resultado final
grupo_perfila = soma_total_elegivel >= 6000

# 4️⃣ Define perfila por linha
df['perfila'] = df['perfil_individual'] & grupo_perfila

# 5️⃣ Exibir DataFrame formatado
from IPython.display import display
print("\n📄 DataFrame completo com colunas 'perfil_individual' e 'perfila':")
display(df)

# 6️⃣ Soma e status final
print(f"\n💰 Soma das dívidas elegíveis individualmente: € {soma_total_elegivel:,.2f}")
print(f"\n🧾 Resultado final: {'✅ PERFILA' if grupo_perfila else '❌ NÃO PERFILA'}")

PDF desbloqueado com sucesso: ./maps/decrypted/decrypted_f9c73ebf-047e-4a20-a4a8-37844b4a6e01.pdf

PDF: decrypted_f9c73ebf-047e-4a20-a4a8-37844b4a6e01.pdf | Total de páginas: 5
Total de produtos detectados em 'decrypted_f9c73ebf-047e-4a20-a4a8-37844b4a6e01.pdf': 7
PDF 'decrypted_f9c73ebf-047e-4a20-a4a8-37844b4a6e01.pdf' movido para: ./maps/decrypted/processed/decrypted_f9c73ebf-047e-4a20-a4a8-37844b4a6e01.pdf

📄 DataFrame completo com colunas 'perfil_individual' e 'perfila':


,bloco_id,nome,nif,mes_mapa,instituicao,divida,litigio,parcela,garantias,num_devedores,prod_financeiro,entrada_incumpr,dat_inicio,dat_fim,perfil_individual,perfila
0,produto_1,RICARDO FILIPE ANTUNES DOS SANTOS,209859636,junho de 2024,"CAIXA DE CRÉDITO AGRÍCOLA MÚTUO DA SERRA DA ESTRELA, CRL (4080)",2094.08,Não,0.00,None,1,Cartão de crédito - com período de free-float,None,2019-06-18,9999-12-31,True,True
1,produto_2,None,None,None,"BNP PARIBAS PERSONAL FINANCE, S.A. - SUCURSAL EM PORTUGAL (0848)",1045.38,Não,62.50,None,1,Cartão de crédito - sem período de free-float,None,2020-12-18,9999-12-31,True,True
2,produto_3,RICARDO FILIPE ANTUNES DOS SANTOS,209859636,junho de 2024,"BNP PARIBAS PERSONAL FINANCE, S.A. - SUCURSAL EM PORTUGAL (0848)",614.63,Não,117.71,None,1,Cartão de crédito - sem período de free-float,None,2021-07-14,9999-12-31,True,True
3,produto_4,None,None,None,COFIDIS (0921),21951.12,Não,518.69,None,1,Crédito pessoal,None,2022-06-24,2029-07-05,True,True
4,produto_5,None,None,None,COFIDIS (0921),287.82,Não,0.00,None,1,Crédito renovável - Linha de crédito,None,2023-04-26,9999-12-31,True,True
5,produto_6,RICARDO FILIPE ANTUNES DOS SANTOS,209859636,junho de 2024,COFIDIS (0921),13000.00,Não,337.65,None,1,Crédito automóvel (excluíndo locações financeiras),None,2024-06-07,2028-12-10,True,True
6,produto_7,RICARDO FILIPE ANTUNES DOS SANTOS,209859636,junho de 2024,"WIZINK BANK, S.A.U. - SUCURSAL EM PORTUGAL (0272)",7366.37,Não,0.00,None,1,Cartão de crédito - com período de free-float,None,2023-02-16,9999-12-31,True,True



💰 Soma das dívidas elegíveis individualmente: € 46,359.40

🧾 Resultado final: ✅ PERFILA


In [140]:
print("\n--- Verificando blocos da CREDIBOM ---")
print(df[df['instituicao'].str.contains("CREDIBOM", case=False, na=False)][['instituicao', 'divida', 'garantias', 'litigio']])


--- Verificando blocos da CREDIBOM ---
                 instituicao   divida  garantias litigio
5  BANCO CREDIBOM, SA (0916)  1248.08       35.7     Não
